In [ ]:
import torch

### Topic 1: Basic Operations and Manipulations

In [ ]:
tensor1 = torch.tensor([[1,2], [3,4]])
tensor2 = torch.tensor([[5,6], [7,8]])

add = tensor1 + tensor2
sub = tensor2 - tensor1
mul = tensor1 * tensor2
div = tensor2 / tensor1

# print(add, sub, mul, div)

mean = torch.mean(tensor1, dtype=float)
maxi = torch.max(tensor1)
s = torch.sum(tensor1)

# print(mean, maxi, s)

reshaped_tensor = tensor1.view(1, -1)
sliced_tensor = tensor1[:, 1]

# print(reshaped_tensor, sliced_tensor)

### Topic 3: Autograd and Automatic Differentiation

In [ ]:
x = torch.tensor(2.0, requires_grad=True)

y = 3 * x ** 2

y.backward()
print(x.grad)

with torch.no_grad():
    z = y * 2

print(z)

In [ ]:
x = torch.tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
y = x * 2 + 1
z = y.mean()

z.backward()
print(x.grad)

### Topic 4: Building Your First Neural Network

In [ ]:
import torch.nn as nn
from torchsummary import summary

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 256)    # Input: 28*28 (digit image pixel dimensions)     Output: 128
        self.fc2 = nn.Linear(256, 128)       # Input: 128     Output: 64
        self.fc3 = nn.Linear(128, 10)        # Input: 64     Output: 10
        self.relu = nn.ReLU()
        self.loss_function = nn.MSELoss()
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    
# net = SimpleNN()
# summary(net, (10,))

### Topic 5: Training and Evaluating Your Model

In [ ]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

features = torch.randn(1000, 28, 28)
lables = torch.randint(0, 9, (1000, 10))

X_train, X_test, y_train, y_test = train_test_split(features, lables, test_size=0.2)

model = SimpleNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

dataset = TensorDataset(X_train, y_train)
batch_size = 32
train_data = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Set the model to training mode
model.train()

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch_features, batch_labels in train_data:
        # Forward Pass
        outputs = model(batch_features)

        # Compute Loss
        loss = model.loss_function(outputs, batch_labels.float())

        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Set the model to evaluation mode
model.eval()

# Evaluation
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = model.loss_function(test_outputs, y_test)

print(f"Test Loss: {test_loss}")

### Topic 6: Fine-Tuning a Pretrained Model

In [ ]:
import torchvision.models as models

# Loading a pretrained model
weights = models.ResNet18_Weights.DEFAULT
pretrained_resnet = models.resnet18(weights=weights)

# Display the model architecture
# print(pretrained_resnet)

# Modify the final layer(s)
num_classes = 10
pretrained_resnet.fc = nn.Linear(pretrained_resnet.fc.in_features, num_classes)

# Freeze or Unfreeze the layer(s)
for param in pretrained_resnet.parameters():
    param.requires_grad = False

# Freeze specific layer(s)
for name, param in pretrained_resnet.named_parameters():
    if 'conv1' in name or 'bn1' in name:
        param.requires_grad = False

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

# Load the MNIST dataset
transform = transforms.Compose([
    # transforms.Grayscale(num_output_channels=3),  # ResNet expects 3 channels
    # transforms.Resize((224, 224)),  # ResNet's input size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
val_dataset = MNIST(root='./data', train=False, transform=transform)

# DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Loading a pretrained model
weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)

# Modify the final classifier (fully connected layers)
# Assuming we're doing a classification task with 10 classes (digits 0-9)
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = SimpleNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and Validation Loop
num_epochs = 10
train_data_size, val_data_size = len(train_loader.dataset), len(val_loader.dataset)
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for batch_data, batch_labels in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_data.size(0)
    train_loss /= train_data_size
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch_features, batch_labels in val_loader:
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            val_loss += loss.item() * batch_features.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == batch_labels).sum().item()
        val_loss /= val_data_size
        val_losses.append(val_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Loss: {train_loss:.4f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Accuracy: {(100 * correct / val_data_size):.2f}%')

torch.save(model, 'digit_model.pth')
    
    # Plot training and validation losses
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()